In [1]:
#pip install fastparquet
#pip install pyarrow

In [2]:
import pyarrow.parquet as pq
import numpy as np
import pandas as pd
import datetime



In [3]:
#pd.show_versions()
#ytd = pd.read_parquet("data/trips/yellow_tripdata_2019-01.parquet",engine="fastparquet")
ytd = pq.read_table("data/trips/yellow_tripdata_2019-01.parquet").to_pandas()
gtd = pq.read_table("data/trips/green_tripdata_2019-01.parquet").to_pandas()
#fhvtd = pq.read_table("data/trips/fhv_tripdata_2019-01.parquet").to_pandas()

Now that the data is read, we need to to some data cleaning.
We start by looking at the columns and removing the columns that aren't needed.

In [4]:
ytd.head()
#fhvtd.head()
#fhvtd.describe()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,1,2019-01-01 00:46:40,2019-01-01 00:53:20,1.0,1.5,1.0,N,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,NaN,None
1,1,2019-01-01 00:59:47,2019-01-01 01:18:59,1.0,2.6,1.0,N,239,246,1,14.0,0.5,0.5,1.00,0.0,0.3,16.30,NaN,None
2,2,2018-12-21 13:48:30,2018-12-21 13:52:40,3.0,0.0,1.0,N,236,236,1,4.5,0.5,0.5,0.00,0.0,0.3,5.80,NaN,None
3,2,2018-11-28 15:52:25,2018-11-28 15:55:45,5.0,0.0,1.0,N,193,193,2,3.5,0.5,0.5,0.00,0.0,0.3,7.55,NaN,None
4,2,2018-11-28 15:56:57,2018-11-28 15:58:33,5.0,0.0,2.0,N,193,193,2,52.0,0.0,0.5,0.00,0.0,0.3,55.55,NaN,None


We only need to keep columns tpep_pickup_datetime, tpep_dropoff_datetime, trip_distance, PULocationID and DOLocationID

In [5]:
ytd = ytd[ytd["fare_amount"]>2.5]
gtd = gtd[gtd["fare_amount"]>2.5]
#fhvtd = fhvtd[fhvtd["fare_amount"]>2.5]

In [6]:
ytd = ytd[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
gtd = gtd[['lpep_pickup_datetime','lpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID']]
#fhvtd = fhvtd[['pickup_datetime','dropOff_datetime','PUlocationID','DOlocationID']]
#fhvtd.dropna(inplace=True)
ytd.head()
#fhvtd.describe()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.5,151,239
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.6,239,246
2,2018-12-21 13:48:30,2018-12-21 13:52:40,0.0,236,236
3,2018-11-28 15:52:25,2018-11-28 15:55:45,0.0,193,193
4,2018-11-28 15:56:57,2018-11-28 15:58:33,0.0,193,193


In [7]:
#fhvtd.head()

To resume the data cleaning, we can convert the pickup and dropoff column to datatime. We can also remove rows where the pickuptime is greater than or equal to the dropoff time. We can also add a column 'trip_time' that says how long the trip lasted in seconds.

In [8]:
ytd['tpep_pickup_datetime'] = pd.to_datetime(ytd['tpep_pickup_datetime'])
ytd['tpep_dropoff_datetime'] = pd.to_datetime(ytd['tpep_dropoff_datetime'])

ytd = ytd.drop(ytd[ytd['tpep_pickup_datetime'] >= ytd['tpep_dropoff_datetime']].index)
ytd['trip_time'] = (ytd['tpep_dropoff_datetime']-ytd['tpep_pickup_datetime']).dt.total_seconds()

gtd['lpep_pickup_datetime'] = pd.to_datetime(gtd['lpep_pickup_datetime'])
gtd['lpep_dropoff_datetime'] = pd.to_datetime(gtd['lpep_dropoff_datetime'])

gtd = gtd.drop(gtd[gtd['lpep_pickup_datetime'] >= gtd['lpep_dropoff_datetime']].index)
gtd['trip_time'] = (gtd['lpep_dropoff_datetime']-gtd['lpep_pickup_datetime']).dt.total_seconds()

# fhvtd['pickup_datetime'] = pd.to_datetime(fhvtd['pickup_datetime'])
# fhvtd['dropOff_datetime'] = pd.to_datetime(fhvtd['dropOff_datetime'])

# fhvtd = fhvtd.drop(fhvtd[fhvtd['pickup_datetime'] >= fhvtd['dropOff_datetime']].index)
# fhvtd['trip_time'] = (fhvtd['dropOff_datetime']-fhvtd['pickup_datetime']).dt.total_seconds()


In [9]:
ytd.head()

,tpep_pickup_datetime,tpep_dropoff_datetime,trip_distance,PULocationID,DOLocationID,trip_time
0,2019-01-01 00:46:40,2019-01-01 00:53:20,1.5,151,239,400.0
1,2019-01-01 00:59:47,2019-01-01 01:18:59,2.6,239,246,1152.0
2,2018-12-21 13:48:30,2018-12-21 13:52:40,0.0,236,236,250.0
3,2018-11-28 15:52:25,2018-11-28 15:55:45,0.0,193,193,200.0
4,2018-11-28 15:56:57,2018-11-28 15:58:33,0.0,193,193,96.0


For convenience sake we can rename the pickup and dropoff columns to 'pickup_time' and 'dropoff_time' respectively. We can also only use time value in total seconds to make it easier to use in the model later.

In [10]:
ytd['tpep_pickup_datetime'] = (ytd['tpep_pickup_datetime']-ytd['tpep_pickup_datetime'].dt.normalize()).dt.total_seconds()
ytd['tpep_dropoff_datetime'] = (ytd['tpep_dropoff_datetime']-ytd['tpep_dropoff_datetime'].dt.normalize()).dt.total_seconds()
ytd.rename(columns = {'tpep_pickup_datetime':'pickup_time','tpep_dropoff_datetime':'dropOff_time'},inplace = True)

gtd['lpep_pickup_datetime'] = (gtd['lpep_pickup_datetime']-gtd['lpep_pickup_datetime'].dt.normalize()).dt.total_seconds()
gtd['lpep_dropoff_datetime'] = (gtd['lpep_dropoff_datetime']-gtd['lpep_dropoff_datetime'].dt.normalize()).dt.total_seconds()
gtd.rename(columns = {'lpep_pickup_datetime':'pickup_time','lpep_dropoff_datetime':'dropOff_time'},inplace = True)

# fhvtd['pickup_datetime'] = (fhvtd['pickup_datetime']-fhvtd['pickup_datetime'].dt.normalize()).dt.total_seconds()
# fhvtd['dropOff_datetime'] = (fhvtd['dropOff_datetime']-fhvtd['dropOff_datetime'].dt.normalize()).dt.total_seconds()
# fhvtd.rename(columns={'PUlocationID':'PULocationID','DOlocationID':'DOLocationID'},inplace=True)
# fhvtd.rename(columns={'dropOff_datetime':'dropOff_time','pickup_datetime':'pickup_time'},inplace=True)


ytd.info()
ytd.describe()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7647363 entries, 0 to 7696614
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   pickup_time    float64
 1   dropOff_time   float64
 2   trip_distance  float64
 3   PULocationID   int64  
 4   DOLocationID   int64  
 5   trip_time      float64
dtypes: float64(4), int64(2)
memory usage: 408.4 MB


,pickup_time,dropOff_time,trip_distance,PULocationID,DOLocationID,trip_time
count,7.647363e+06,7.647363e+06,7.647363e+06,7.647363e+06,7.647363e+06,7.647363e+06
mean,5.143478e+04,5.163587e+04,2.844372e+00,1.653884e+02,1.635624e+02,9.991236e+02
std,2.137829e+04,2.161727e+04,3.774781e+00,6.640864e+01,7.038317e+01,4.517151e+03
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,3.623100e+04,3.652200e+04,9.200000e-01,1.260000e+02,1.130000e+02,3.700000e+02
50%,5.376800e+04,5.416700e+04,1.550000e+00,1.620000e+02,1.620000e+02,6.140000e+02
75%,6.866700e+04,6.907000e+04,2.850000e+00,2.340000e+02,2.340000e+02,1.003000e+03
max,8.639900e+04,8.639900e+04,8.318000e+02,2.650000e+02,2.650000e+02,2.618881e+06


In the taxi_zone_lookup file, zones 264 and 265 are marked as unknown so we remove this from our dataset. We also remove trip_distances less than 0.0 miles aswell as trips that lasted more than roughly 5,5 hours.

In [11]:
def cleanValues(df):
    df = df[(df.PULocationID < 264) & (df.PULocationID > 0)]
    df = df[(df.DOLocationID < 264) & (df.DOLocationID > 0)]
    df = df[df.trip_time != 0]
    df = df[df.trip_time < 20000]
    df = df[df.trip_distance != 0.0]
    return df;

ytd = cleanValues(ytd)
gtd = cleanValues(gtd)
#fhvtd = cleanValues(fhvtd)
# ytd.apply(cleanValues(ytd))
# gtd.apply(cleanValues(gtd))
# fhvtd.apply(cleanValues(fhvtd))

#combined = pd.concat([ytd,gtd,fhvtd],ignore_index=True)

# ytd = ytd[ytd.PULocationID < 264]
# ytd = ytd[ytd.DOLocationID < 264]
# ytd = ytd[ytd.trip_time != 0]
#ytd = ytd[ytd.trip_distance != 0.0]
#gtd = gtd[gtd.trip_distance != 0.0]
# ytd=ytd[ytd.trip_time<20000]
ytd.info()
ytd.describe()
combined = pd.concat([ytd,gtd],ignore_index=True, axis=0)
combined.describe()
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7429821 entries, 0 to 7696614
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   pickup_time    float64
 1   dropOff_time   float64
 2   trip_distance  float64
 3   PULocationID   int64  
 4   DOLocationID   int64  
 5   trip_time      float64
dtypes: float64(4), int64(2)
memory usage: 396.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8080080 entries, 0 to 8080079
Data columns (total 6 columns):
 #   Column         Dtype  
---  ------         -----  
 0   pickup_time    float64
 1   dropOff_time   float64
 2   trip_distance  float64
 3   PULocationID   int64  
 4   DOLocationID   int64  
 5   trip_time      float64
dtypes: float64(4), int64(2)
memory usage: 369.9 MB


We can now check if there are any missing values in our dataset.

In [12]:
def missing_cols(df):
    '''prints out columns with its amount of missing values'''
    total = 0
    for col in df.columns:
        missing_vals = df[col].isnull().sum()
        total += missing_vals
        pct = df[col].isna().mean() * 100
        if missing_vals != 0:
            print(f"{col} => {df[col].isnull().sum()},{round(pct,2)}%")
    
    if total == 0:
        print("no missing values left")
            
#missing_cols(ytd)
#missing_cols(fhvtd)
#missing_cols(gtd)
missing_cols(combined)

no missing values left


In [13]:
ytd.describe()
corr = ytd["trip_time"].corr(ytd["trip_distance"])
print(corr)

0.7897065624124308


In [15]:

from sklearn.model_selection import train_test_split

copy = ytd.copy()
copy["pickup_time"] = (ytd[("pickup_time")] - ytd[("pickup_time")].mean()) / ytd[("pickup_time")].std() 
copy["trip_distance"] = (ytd[("trip_distance")] - ytd[("trip_distance")].mean()) / ytd[("trip_distance")].std()
copy["PULocationID"] = (ytd[("PULocationID")] - ytd[("PULocationID")].mean()) / ytd[("PULocationID")].std()
copy["DOLocationID"] = (ytd[("DOLocationID")] - ytd[("DOLocationID")].mean()) / ytd[("DOLocationID")].std()


copy["trip_time"] = (ytd[("trip_time")] - ytd[("trip_time")].mean()) / ytd[("trip_time")].std() 

#X = np.asarray(copy[["pickup_time","trip_distance","PULocationID","DOLocationID"]])
#Y = np.asarray(copy["trip_time"])

X = np.asarray(combined[["pickup_time","trip_distance", "PULocationID", "DOLocationID"]])
Y = np.asarray(combined["trip_time"])

X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.33,random_state = 1)

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
lm = lm.fit(X_train, Y_train)
Y_hat = lm.predict(X_test)
print("Linear Regression:")
print("Accuracy: ",r2_score(Y_test, Y_hat))
print("Error: ",mean_squared_error(Y_test, Y_hat))

from sklearn.ensemble import RandomForestRegressor

# f = RandomForestRegressor()
# f = f.fit(X_train, Y_train)
# Y_hat = t.predict(X_test)
# print("Random Forest Regressor:")
# print("Accuracy: ",r2_score(Y_test, Y_hat))
# print("Error: ",mean_squared_error(Y_test, Y_hat))



# X = ytd[['trip_distance','pickup_time']]
# Y = ytd['trip_time']
# X = X.values.reshape(len(X), 2)
# Y = Y.values.reshape(len(Y), 1)

# X_train = X[:int(0.8*len(X))]
# X_test = X[int(0.8*len(X)):]

# Y_train = Y[:int(0.8*len(Y))]
# Y_test = Y[int(0.8*len(Y)):]

# from sklearn.tree import DecisionTreeRegressor

# t = DecisionTreeRegressor()
# t = t.fit(X_train, Y_train)
# Y_hat = t.predict(X_test)
# print(r2_score(Y_test, Y_hat))

# from sklearn.metrics import mean_squared_error, r2_score
# from sklearn.linear_model import LinearRegression
# lm = LinearRegression()
# lm = lm.fit(X_train, Y_train)
# Y_hat = lm.predict(X_test)
# print(r2_score(Y_test, Y_hat))
# print(mean_squared_error(Y_test, Y_hat))

Linear Regression:
Accuracy:  0.6356506715451596
Error:  144852.89672964992
